In [8]:
import random
import numpy as np
import tensorflow as tf

In [3]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')


text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()


text = text[300000:800000]


characters = sorted(set(text))


char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))


1115394/1115394 [==============================] - 0s 0us/step


In [4]:
SEQ_LENGTH = 40
STEP_SIZE = 3


sentences = []
next_char = []


In [5]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])


x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)


for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1


<ipython-input-5-4feb3c307947>:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)
<ipython-input-5-4feb3c307947>:9: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)


In [7]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM


model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))


model.fit(x, y, batch_size=256, epochs=4)



def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Epoch 1/4
651/651 [==============================] - 126s 190ms/step - loss: 2.6975
Epoch 2/4
651/651 [==============================] - 117s 180ms/step - loss: 2.3058
Epoch 3/4
651/651 [==============================] - 116s 179ms/step - loss: 2.1929
Epoch 4/4
651/651 [==============================] - 112s 173ms/step - loss: 2.1124


In [9]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1


        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]


        generated += next_character
        sentence = sentence[1:] + next_character
    return generated


In [10]:
print(generate_text(300, 0.2))

ry: though
i have for the most part been the the the the the the stard the mand the to the the the the mand the sould the beat the sored with the fore so the hand the the the the to mand the the the the the the the the har sourd the the the the the soust the dore the the the the cand the the care the the the the the here the pares the sor
